In [87]:
# Import libraries
import os
import pandas as pd
from sqlalchemy import create_engine, inspect, MetaData, Table, select, func, case, cast, Numeric
from local_settings import postgresql as settings
import matplotlib.pyplot as plt
import plotly.express as px

In [8]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('172.28.240.1', 5432))
print("Port is open" if result == 0 else "Port is closed")
sock.close()


Port is open


In [10]:
# Define a function for creating the sqlalchemy engine
def get_engine(user, passwd, host, port, db):
    url = f"postgresql+psycopg2://{user}:{passwd}@{host}:{port}/{db}"
    engine = create_engine(url, pool_size= 50, echo= False)

    return engine

# Create the engine and respective connection
engine = get_engine(
    user= settings['pguser'],
    passwd= settings['pgpasswd'],
    host= settings['pghost'],
    port= settings['pgport'],
    db= settings['pgdb']
)

connection = engine.connect()


Once the engine is connected to the remote database, we can use the `inspect` function from SQLAlchemy to examine the contents of the database, i.e. its tables.

In [46]:
path = os.getcwd() + '/food_carbon_footprint_2018_long.csv'
foodprint_2018 = pd.read_csv(path, sep= ';')
foodprint_2018.to_sql(name= 'foodprint', con= connection, if_exists= 'replace', index= False)

250

In [5]:
inspector = inspect(engine)

print(inspector.get_table_names())

['foodprint']


In [11]:
metadata= MetaData()
foodprint = Table('foodprint', metadata, autoload_replace= True, autoload_with= engine)

In [12]:
# Define a function for retrieving the result proxy

def fetch_query_results(input_type, input):
    """
    Fetch query results as a pandas DataFrame.

    Parameters:
    ----------
    input_type : str
        'stmt' for a SQLAlchemy statement, or 'query' for a raw SQL string.
    input_value : object
        SQLAlchemy statement or raw SQL string depending on input_type.

    Returns:
    -------
    pd.DataFrame
        Query results as a DataFrame.
    """
    
    if input_type == 'stmt':
        results = connection.execute(input).fetchall()
        return pd.DataFrame(results)
    elif input_type == 'query':
        return pd.read_sql_query(input, connection)
    else:
        print("Invalid input_type. Please choose 'stmt' for SQLAlchemy statements or 'query' for raw SQL strings.")

In [13]:
stmt = select(foodprint)

print(stmt, '\n')

fetch_query_results(input_type= 'stmt', input= stmt).head(5)

SELECT foodprint.country, foodprint.product, foodprint.metric, foodprint.value 
FROM foodprint 



,country,product,metric,value
0,Argentina,Pork,Supplied for Consumption (kg/person/year),10.51
1,Australia,Pork,Supplied for Consumption (kg/person/year),24.14
2,Albania,Pork,Supplied for Consumption (kg/person/year),10.88
3,Iceland,Pork,Supplied for Consumption (kg/person/year),21.69
4,New Zealand,Pork,Supplied for Consumption (kg/person/year),22.29


In [14]:
# Dimensions of the foodprint table
stmt = select(func.count()).select_from(foodprint)

print(f"({connection.execute(stmt).scalar()}, {len(foodprint.columns)})")

(3250, 4)


In [15]:
# Data type of each column
for column in foodprint.columns:
    print(f"{column.key}: {column.type}")

country: TEXT
product: TEXT
metric: TEXT
value: DOUBLE PRECISION


In [16]:
# Missing data in each column
print("Missing values: \n")
for column in foodprint.columns:
    stmt = select(func.count()).where(column.is_(None))
    result = connection.execute(stmt).scalar()
    print(f"{column.key}: {result}")

Missing values: 

country: 0
product: 0
metric: 0
value: 0


In [17]:
# Number of countries under analysis for each product and metric

query = """
    SELECT product,
           metric,
           COUNT(country) AS number_of_countries
        FROM foodprint
        GROUP BY product, metric
        ORDER BY product, metric
"""

fetch_query_results('query', query)


,product,metric,number_of_countries
0,Animal products,Total Kg CO2/year/person,130
1,Beef,Kg CO2/person/year,130
2,Beef,Supplied for Consumption (kg/person/year),130
3,Difference Animal Non-Animals,Products/person/year in kg,130
4,Eggs,Kg CO2/person/year,130
5,Eggs,Supplied for Consumption (kg/person/year),130
6,Fish,Kg CO2/person/year,130
7,Fish,Supplied for Consumption (kg/person/year),130
8,Lamb & Goat,Kg CO2/person/year,130
9,Lamb & Goat,Supplied for Consumption (kg/person/year),130


In the `product` and `metric` fields, there are a few values that do not correspond directly to individual items in these categories, as they aggregate information at a global level. Specifically, the values `Animal products`, `Non-animal products` and `Difference Animal Non-Animals` appear in the `product` field. The first two are associated with the metric `Total Kg CO₂/year/person` in the `metric` field, and the latter to a `Products/person/year in kg` metric.

To address this particular feature of the dataset, it can be transformed by filtering out these values and then creating a new categorical field that distinguishes between the `Animal` and `Non-animal` product categories. Regarding the `Difference Animal Non-Animals` item, it can be assessed based on these two categories in the new field.

In [18]:
animal = ['Beef', 'Eggs', 'Fish', 'Lamb & Goat', 'Milk - inc. Cheese', 'Pork', 'Poultry']
non_animal = ['Wheat & Wheat Products', 'Rice', 'Soybeans', 'Nuts inc. Peanut Butter']
filter_out = ['Animal products', 'Non-animal products', 'Difference Animal Non-Animals']

stmt = select(foodprint.columns.country,
              foodprint.columns.product,
              case(
                    (foodprint.columns.product.in_(animal), 'Animal'),
                    (foodprint.columns.product.in_(non_animal), 'Non-animal'),
                    else_= None).label('product_category'),
              foodprint.columns.metric,
              foodprint.columns.value
)

stmt = stmt.where(foodprint.columns.product.not_in(filter_out))

print(stmt, '\n')

fetch_query_results('stmt', stmt).head()

SELECT foodprint.country, foodprint.product, CASE WHEN (foodprint.product IN (__[POSTCOMPILE_product_1])) THEN :param_1 WHEN (foodprint.product IN (__[POSTCOMPILE_product_2])) THEN :param_2 END AS product_category, foodprint.metric, foodprint.value 
FROM foodprint 
WHERE (foodprint.product NOT IN (__[POSTCOMPILE_product_3])) 



,country,product,product_category,metric,value
0,Argentina,Pork,Animal,Supplied for Consumption (kg/person/year),10.51
1,Australia,Pork,Animal,Supplied for Consumption (kg/person/year),24.14
2,Albania,Pork,Animal,Supplied for Consumption (kg/person/year),10.88
3,Iceland,Pork,Animal,Supplied for Consumption (kg/person/year),21.69
4,New Zealand,Pork,Animal,Supplied for Consumption (kg/person/year),22.29


In [12]:
# Update the foodprint table in the remote database
(
    fetch_query_results('stmt', stmt)
    .to_sql("foodprint1", engine, if_exists= 'replace', index= False, chunksize= 1000, method= 'multi')
)

2860

In [19]:
# Inspect the existence of foodprint_bkp in the remote database
inspector = inspect(engine)
print(inspector.get_table_names())


['foodprint1', 'foodprint']


After making sure of the existence of the original and the altered table in the `foodprint_2018` remote database, instantiate the `foodprint1` table in the SQLAlchemy context, and take a look at the first 10 rows:

In [25]:
foodprint1 = Table('foodprint1', metadata, autoload_replace= True, autoload_with= engine)

query = """
    SELECT *
        FROM foodprint1
        LIMIT 10
"""

fetch_query_results('query', query)

,country,product,product_category,metric,value
0,Argentina,Pork,Animal,Supplied for Consumption (kg/person/year),10.51
1,Australia,Pork,Animal,Supplied for Consumption (kg/person/year),24.14
2,Albania,Pork,Animal,Supplied for Consumption (kg/person/year),10.88
3,Iceland,Pork,Animal,Supplied for Consumption (kg/person/year),21.69
4,New Zealand,Pork,Animal,Supplied for Consumption (kg/person/year),22.29
5,USA,Pork,Animal,Supplied for Consumption (kg/person/year),27.64
6,Uruguay,Pork,Animal,Supplied for Consumption (kg/person/year),16.84
7,Luxembourg,Pork,Animal,Supplied for Consumption (kg/person/year),43.58
8,Brazil,Pork,Animal,Supplied for Consumption (kg/person/year),12.60
9,Kazakhstan,Pork,Animal,Supplied for Consumption (kg/person/year),10.36


In [95]:
query = """
    SELECT  product_category
           ,metric
           ,SUM(value) AS Total
        FROM foodprint1
        GROUP BY product_category, metric  
"""

fetch_query_results('query', query)

,product_category,metric,total
0,Animal,Supplied for Consumption (kg/person/year),26427.96
1,Non-animal,Kg CO2/person/year,7663.03
2,Non-animal,Supplied for Consumption (kg/person/year),13769.92
3,Animal,Kg CO2/person/year,98706.08


In [68]:
# Metrics comparison for both categories

stmt = select(foodprint1.columns.product_category,
              foodprint1.columns.metric,
              func.sum(foodprint1.columns.value).label('Total'))

stmt = stmt.group_by(foodprint1.columns.product_category, foodprint1.columns.metric)
stmt = stmt.order_by(foodprint1.columns.product_category, foodprint1.columns.metric)

print(stmt, ' \n')

df = fetch_query_results('stmt', stmt)
df['Total'] = pd.to_numeric(df['Total'], errors= 'coerce')
df

SELECT foodprint1.product_category, foodprint1.metric, sum(foodprint1.value) AS "Total" 
FROM foodprint1 GROUP BY foodprint1.product_category, foodprint1.metric ORDER BY foodprint1.product_category, foodprint1.metric  



,product_category,metric,Total
0,Animal,Kg CO2/person/year,98706.08
1,Animal,Supplied for Consumption (kg/person/year),26427.96
2,Non-animal,Kg CO2/person/year,7663.03
3,Non-animal,Supplied for Consumption (kg/person/year),13769.92


In [91]:
connection.rollback()

In [89]:
stmt = select(foodprint1.columns.country,
              foodprint1.columns.product_category,
              foodprint1.columns.metric,
              cast(foodprint1.columns.value, Numeric(10,4)).label('value'))

df = fetch_query_results('stmt', stmt)

In [94]:
df_co2 = df[df['metric'] == 'Kg CO2/person/year']

# Create boxplot for CO2
fig_co2 = px.box(
    df_co2,
    x='product_category',
    y='value',
    color='product_category',
    title='Kg CO₂/person/year by Product Category',
    labels={'Total': 'Kg CO₂/person/year', 'product_category': 'Category'},
    hover_data= ['country']
)

fig_co2.show()

In [63]:
query = """
    SELECT product_category,
           metric,
           MIN(value) AS min,
           MAX(value) AS max
    FROM foodprint1
    GROUP BY product_category, metric
"""


fetch_query_results('query', query)

,product_category,metric,min,max
0,Animal,Supplied for Consumption (kg/person/year),0.0,430.76
1,Non-animal,Kg CO2/person/year,0.0,219.76
2,Non-animal,Supplied for Consumption (kg/person/year),0.0,197.50
3,Animal,Kg CO2/person/year,0.0,1712.00


In [65]:
query = """
    SELECT country
           product_Category,
           metric,
           value
        FROM foodprint1
        WHERE value = 0
"""

fetch_query_results('query', query)

,product_category,metric,value
0,Kuwait,Supplied for Consumption (kg/person/year),0.0
1,United Arab Emirates,Supplied for Consumption (kg/person/year),0.0
2,Algeria,Supplied for Consumption (kg/person/year),0.0
3,Pakistan,Supplied for Consumption (kg/person/year),0.0
4,Saudi Arabia,Supplied for Consumption (kg/person/year),0.0
...,...,...,...
70,Ghana,Kg CO2/person/year,0.0
71,Togo,Kg CO2/person/year,0.0
72,Gambia,Kg CO2/person/year,0.0
73,Sierra Leone,Kg CO2/person/year,0.0
